# Streamflow Catalogue Demonstration: Records organization

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to organize most of the downloaded/received streamflow records in a adequate way prior to the data processing. 

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* numpy
* os
* pandas
* tqdm
* tabula

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* 

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References
- Please check the Streamflow Catalogue for full information about how to reference the streamflow records. 

## Observations
- This notebook encompasses 27 (out of 38) countries/regions covered in EStreams. The reason is that for some providers, the data was processed in a non-elegant way. Users can expect updates, or can simply adapt the current codes for their needs.
- Hence, users should use this notebook as a guidance when processing their own data, and are therefore invited to adapt, adjust and create new scripts based on the current available codes. 

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import tqdm as tqdm
import glob
import os
import tabula

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here. 

In [ ]:
# Set the directory:
os.chdir(PATH)

# Import and process the data

### 1. Austria

In [ ]:
# First, we specify the path to the folder containing our CSV files:
path = r'data/streamflow/raw_data/AT/Q-Tagesmittel'

# Next, we list all the filenames in the folder:
filenames = glob.glob(path + "/*.csv")

# Create an empty DataFrame to store the final time series data:
timeseries_AT = pd.DataFrame(index=pd.date_range('01-01-1900', '12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # Read the CSV file to determine at which row the dataset starts:
    aux_1 = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows=0, delimiter=";", skip_blank_lines=False)
    row_start = int(aux_1.index[aux_1.iloc[:, 0] == "Werte:"].tolist()[0])

    # Read the CSV file, skipping unnecessary rows:
    data = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows=row_start, delimiter=";", decimal=",")

    # Reset index and set dates as index:
    data.reset_index(inplace=True)
    data["dates"] = pd.to_datetime(data["Werte:"], format='%d.%m.%Y %H:%M:%S')
    data.set_index("dates", inplace=True)
    data.drop("Werte:", axis=1, inplace=True)
    data.columns = ["Q_m3_s"]

    # Convert non-numeric values to NaN:
    data["Q_m3_s"] = data["Q_m3_s"].replace(" Lücke", np.nan)

    # Remove whitespace from column contents:
    data['Q_m3_s'] = data['Q_m3_s'].str.strip()

    # Convert data to numeric:
    data["Q_m3_s"] = data["Q_m3_s"].apply(float)
    
    # Retrieve the station name from the filename:
    namestation = os.path.basename(filename)
    namestation = namestation.split("-", 2)[2]
    namestation = namestation.replace(".csv", "")
    
    # Store the data in the final DataFrame, using the station name as a column label:
    timeseries_AT.loc[:, int(namestation)] = data.Q_m3_s

# Check the final dataset out
timeseries_AT.head()

### 2. Bosnia and H.
- Here we actually convert data from yearly hydrological reports in PDF to CSV.
- We only used the yearbooks from 1987-2019, since before then, the PDF-files were not possible to be converted to CSV-files.
- Observe that there are several steps that are manually adjusted (checked) to make this current code work, such as retrieving the pages with valid information.
- For the steps manually adjusted, we make sure to leave the label (manually checked) in order to guide potential users.

In [ ]:
## BA records (PDF first):
path =r'data/streamflow/raw_data/BA'
filenames = glob.glob(path + "/*.pdf")

### Convertion from PDF to CSV:
# Here we specify the pages where there is data in each PDF file (manual inspection):
# The pdfs should be in chronological order (1987-2019):
pages_pdfs = ["56-83", "55-83", "56-84", "56-84", "53-78", "47-63", "51-67", "55-73", "91-125",
              "91-129", "101-146", "109-152", "117-158", "120-158", "139-189", "145-198", "151-209", "151-213", 
              "162-222", "162-230", "168-242", "168-248", "170-254", "166-250"]

i = 0
for filename in tqdm.tqdm(filenames):
    
    # Specify the output path for the Excel file
    excel_path = filename.replace(".pdf", ".csv")

    # Convert the PDF to Excel
    tabula.convert_into(filename, excel_path, output_format='csv', pages=pages_pdfs[i])
    
    i = i + 1

### Extraction of the time series
path =r'data/streamflow/raw_data/BA'
filenames = glob.glob(path + "/*.csv")

# Here we create a full dataframe for the entire time-series:
timeseries_BA = pd.DataFrame(index = pd.date_range('01-01-1987','12-31-2019', freq='D'))

# Here we choose only the rows related to the days to keep in our time-series dataframes:
rows_to_keep =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 
                 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
rows_to_keep = list(map(str, rows_to_keep))

# This is the sep type (manually checked):
sep_type = [";", ",", ";", ",", ",", ",", ",", ",", ";", ";", ";", ",", ";", ",", ";", ";", ";", ",", ";", ";", ";", ";", ";", ";"]

i = 0

for filename_net in tqdm.tqdm(filenames_net):
        
    filename_ts = filenames[i]
    
    # Here we read the time-series:
    timeseries = pd.read_csv(filename_ts, skiprows=0, header = 0, decimal=',', sep = sep_type[i])
    
    # Here we read the stations for the time-series:
    stations_df = pd.read_csv(filename_net,  encoding='unicode_escape', sep= ";")
    
    # Replace non-properly read characters with '?'
    stations_df = stations_df.astype(str)
    stations_df = stations_df.apply(lambda x: x.str.encode('unicode_escape', 'replace').str.decode('utf-8'))
    stations = stations_df.iloc[:, 1].tolist()
    
    # Now we select only the rows that refer to days to keep:
    timeseries = timeseries[timeseries.iloc[:, 0].isin(rows_to_keep)]
    timeseries.reset_index(inplace=True, drop=True)
    
    # First we ensure that all is an object:
    timeseries = timeseries.astype(str)

    # Now we convert to float after replacing any comma to point:
    timeseries = timeseries.applymap(lambda x: float(x.replace(',', '.')))
    
    # Here we adjust the column names:
    timeseries.columns = ["Day", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
    
    # Here we get the year:
    year_station = os.path.basename(filename_ts)
    year_station = year_station.split(" ", 2)[0]

    # This part is done for each date:
    timeseries_numpy = timeseries.iloc[:, :].to_numpy()
    timeseries_aux_df = pd.DataFrame(index = range(31*12), columns =stations)
    timeseries_aux_df["dates"] = np.nan


    # First we do the loop at each month:
    rows_dia_aux = 0
    for mes in range(1, 13):
        row_ts_aux = 0
    
        # Now we do at each station:
        k = 0
        for station in stations:
        
            for dia in range(31):
            
                # Here we insert the time-series value:
                timeseries_aux_df.iloc[rows_dia_aux + dia, k] = timeseries_numpy[row_ts_aux, mes]

                # Here we insert the "date":
                timeseries_aux_df.iloc[rows_dia_aux + dia, -1] = str(dia + 1) + "-" + str(mes) + "-" + str(year_station)
        
                # Here we get the rows for getting the data:
                row_ts_aux = row_ts_aux + 1
        
            # Here it is the column:
            k = k + 1
        

        rows_dia_aux = rows_dia_aux + 31    
    

    # At this part we convert our previously written datetime column to properlly a datetime type. 
    # However, we have to use the "try" to deal with the rows that do not present real dates (e.g., 31.02.2022): 

    for dia in range(len(timeseries_aux_df)):    
        try:
            timeseries_aux_df.loc[dia, "dates"] = pd.to_datetime(timeseries_aux_df.iloc[dia, -1], format='%d-%m-%Y')
    
        except:
            timeseries_aux_df.loc[dia, "dates"] = np.nan
        
    ## Here we delete the rows that do not present real dates (e.g., 31.02.2022): 
    timeseries_aux_df = timeseries_aux_df.dropna(subset=['dates'])
    timeseries_aux_df.set_index('dates', inplace = True)

    # Convert non-numeric values to NaN for the entire DataFrame
    timeseries_aux_df = timeseries_aux_df.apply(pd.to_numeric, errors='coerce')
    
    
    # Here we assign the value of the year at the correct location:
    timeseries_BA.loc[timeseries_aux_df.index, timeseries_aux_df.columns] = timeseries_aux_df
    
    i = i + 1

# Check the final dataset out
timeseries_BA.head()

### 3. GRDC data 
- Countries used: BG, BY, CY, EE, GR, HU, IT, LT, LV, MD, MK, RO, RS, RU, SK, TR, UA

In [ ]:
## GRDC network:
# Read the data
network_GRDC = pd.read_excel(r'data/streamflow/raw_data/GRDC/GRDC_Stations.xlsx', skiprows=0)

# Filter only Europe (wmo_reg == 6)
network_GRDC = network_GRDC[network_GRDC.wmo_reg == 6]

## GRDC time series
# First we can read our files within the folder:
path =r'data/raw_data/GRDC'
filenames = glob.glob(path + "/*.txt")

timeseries_GRDC = pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # First we read our time-series:
    data = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows = 36, delimiter = ";", usecols = [0, 2])

    # We define new column names:
    names = ['dates', 'Qm3s']
    data.columns = names

    # Convert our column of dates to datetime format:
    data["dates"] = pd.to_datetime(data["dates"], format='%Y-%m-%d')

    # We can replace the -999.0 to np.nan:
    data.Qm3s.replace(-999.0, np.nan, inplace=True)

    # Here we can set the index as the dates column:
    data.set_index("dates", inplace = True)

    # First we can retrieve the station name:
    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 1)[0]
    
    timeseries_GRDC.loc[:, int(namestation)] = data.Qm3s

# Check the final time series:
timeseries_GRDC.head()

### 4. Switzerland

In [ ]:
## CAMELS-CH network:
# Read the data
network_CH = pd.read_csv(r'data/streamflow/raw_data/CH/CAMELS_CH_topographic_attributes.csv', skiprows=1, encoding='latin-1', sep= ";")
network_CH = network_CH.set_index("gauge_id")

## CAMELS-CH time series
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/CH'
filenames = glob.glob(path + "/*.csv")

timeseries_CH = pd.DataFrame(index = pd.date_range('01-01-1981','12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # First we read our time-series:
    data = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows = 0, delimiter = ";", usecols = [0, 1, 2])

    # We define new column names:
    names = ['dates', 'Qm3s', "Qmmday"]
    data.columns = names

    # Convert our column of dates to datetime format:
    data["dates"] = pd.to_datetime(data["dates"], format='%Y-%m-%d')

    # Here we can set the index as the dates column:
    data.set_index("dates", inplace = True)

    # First we can retrieve the station name:
    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 5)[4]
    namestation = namestation.replace(".csv", "")
    timeseries_CH.loc[:, int(namestation)] = data.Qm3s

# Check the final time series:
timeseries_CH.head()

### 5. Spain

In [ ]:
## Spanish time series
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/ES'
filenames = glob.glob(path + "/*/")

timeseries_ES = pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2022', freq='D'))
i = 0
for filename in tqdm.tqdm(filenames):
    
    ## This part is regardig the network information data:
    network_filename = pd.read_csv(filename + "estaf.csv", encoding='latin-1', skiprows=0, 
                          usecols = ['indroea', 'lugar', 'suprest', 'suprcnc', 'alti', 'num_cuenca',
                                     'long', 'lat', 'xetrs89', 'yetrs89'], 
                      delimiter = ";")
    
    # Now the concatatanion is made to get the full network:
    if i == 0:
        network = network_filename
    else:
        network = pd.concat([network, network_filename], axis=0)
    i = i + 1
    
    ## This part is regarding the streamflow data:
    timespain = pd.read_csv(filename + "afliq.csv", encoding='latin-1', delimiter = ";")

    names = ['ID', 'date', 'height_m', 'Q_m3_s']

    timespain.columns = names
    timespain["date"] = pd.to_datetime(timespain["date"], format='%d/%m/%Y')
    

    stationsspain = timespain["ID"].unique().tolist()
    
    #print(len(stationsspain)) # Just to check the number of stations available at each dataset
    
    # Here we fill our streamflow datasets into the single dataframe:
    for station in stationsspain:

        
        timespain_station = timespain[timespain.ID == station]
        timespain_station.set_index('date', inplace = True)

        timeseries_ES.loc[:, station] = timespain_station.Q_m3_s

# Check the final time series:
timeseries_ES.head()

### 6. France

In [ ]:
## French time series
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/FR'
filenames = glob.glob(path + "/*.csv")

timeseries_FR = pd.DataFrame(index = pd.date_range('01-01-1840','12-31-2023', freq='D'))
timeseries_quality_FR = pd.DataFrame(index = pd.date_range('01-01-1840','12-31-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):

    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 1)[0]
    
    data = pd.read_csv(filename, encoding='latin-1', engine='python', usecols = ['Date (TU)', 'Valeur (en mÂ³/s)', 
                      'Qualification'])

    names = ['dates', 'Qm3s', 'Status']
    data.columns = names

    data["dates"].replace({'T': ' '}, regex=True, inplace=True)
    data["dates"].replace({'Z': ' '}, regex=True, inplace=True)

    data["dates"] = pd.to_datetime(data['dates'], format='%Y/%m/%d')
    data.set_index("dates", inplace = True)
    
    timeseries_FR.loc[:, str(namestation)] = data.Qm3s
    timeseries_quality_FR.loc[:, str(namestation)] = data.Status

# Check the final time series:
timeseries_FR.head()

### 7. United kingdom (GB & NI)

In [ ]:
## UK time series
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/UK'
filenames = glob.glob(path + "/*.csv")

timeseries_UK = pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    
    # The number of rows before the real streamflow data is variable, then we can solve it as:
    # First we select the row where we have the second column with the value "last", and we know that the stremflow starts
    # right after that:
    dataframefull = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows = 0, delimiter = ",", header = None)
    rownnumber = dataframefull.index[dataframefull[1] == "last"].tolist()[0]
    
    # Now we read our time-series:
    data = pd.read_csv(filename, encoding='latin-1', engine='python', skiprows = rownnumber + 1, 
                       delimiter = ",", usecols = [0, 1], names = ["dates", "Qm3s"])

    # Convert our column of dates to datetime format:
    data["dates"] = pd.to_datetime(data["dates"], format='%Y-%m-%d')

    # Here we can set the index as the dates column:
    data.set_index("dates", inplace = True)

    # Here we can retrieve the station name:
    namestation = dataframefull.iloc[3, 2]
    
    timeseries_UK.loc[:, int(namestation)] = data.Qm3s

# Check the final time series:
timeseries_UK.head()

### 8. Greece
- HCMR

In [ ]:
## Greek time series from HCMR
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/GR/HCMR'
filenames = glob.glob(path + "/*.csv")

timeseries_GRHCMR = pd.DataFrame(index = pd.date_range('01-01-2014','12-31-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    namestation = os.path.basename(filename)
    namestation = namestation.split("-", 3)[1]

    # Here you can get rid of the space before the variable name:
    namestation = namestation.lstrip()

    # Here we read our data:
    timeseries_filename = pd.read_csv(filename, encoding='latin-1', header = 1, delimiter = ",", 
                                   decimal = ".",
                                   names = [ "dates", "Q_m3s"])

    # Here we replace the "cms" to nothing at the time-series:
    timeseries_filename['Q_m3s'] = timeseries_filename['Q_m3s'].str.replace(' cms', '')

    # Now we convert our dataset to datetime:
    timeseries_filename["dates"] = pd.to_datetime(timeseries_filename["dates"], format='%Y-%m-%d %H:%M:%S')

    timeseries_filename.set_index("dates", inplace = True)

    # Convert the 'your_column_name' column to numeric
    timeseries_filename['Q_m3s'] = pd.to_numeric(timeseries_filename['Q_m3s'], errors='coerce')

    # Resample the data to daily intervals, taking the mean of each day
    timeseries_filename = timeseries_filename.resample('D').mean()
    
    timeseries_GRHCMR.loc[:, namestation] = timeseries_filename.Q_m3s

# Check the final time series:
timeseries_GRHCMR.head()

### 9. Ireland
- EPA

In [ ]:
## Irish time series from EPA
path =r'data/streamflow/raw_data/IE/EPA'
filenames = glob.glob(path + "/*.csv")

timeseries_IEEPA = pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in filenames:
    # Here we retrieve only the station code:
    dataframefull = pd.read_csv(filename,  delimiter = ";", nrows = 3, header = None)
    station = dataframefull.iloc[1, 1]
    
    # It seems that we download some stage as well, so we have to get rid of those:
    if dataframefull.iloc[2, 1] != "River Discharge":
        pass
    
    else:
        # Now we read our time-series:
        dataf = pd.read_csv(filename, skiprows = 7, delimiter = ";", usecols=range(3), names = ["dates", "Qm3s", "quality"])
    
        # Convert our column of dates to datetime format:
        dataf["dates"] = pd.to_datetime(dataf["dates"], format='%Y-%m-%d')
        dataf.set_index("dates", inplace = True)
        data = dataf.resample('D').mean()
    
        data["quality"] = dataf.quality.values
    
        timeseries_IEEPA.loc[:, str(station)] = data.Qm3s

# Check the final time series:
timeseries_IEEPA.head()        

- OPW

In [ ]:
## Irish time series from OPW
# Data lecture
path =r'data/streamflow/raw_data/IE/OPW'
filenames = glob.glob(path + "/*.txt")

timeseries_IEOPW= pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in filenames:
    # Here we retrieve only the station code:
    dataframefull = pd.read_csv(filename,  delimiter = "\t", nrows = 6, header = None)
    station = str(dataframefull.iloc[1, 1])
    
    # It seems that we download some stage as well, so we have to get rid of those:
    if dataframefull.iloc[5, 1] != "cubic meter per second":
        1 + 1
    
    else:
        # Now we read our time-series:
        data = pd.read_csv(filename, skiprows = 8, delimiter = "\t", usecols=range(3), names = ["dates", "Qm3s", "quality"])
        
        # Convert our column of dates to datetime format:
        data["dates"] = pd.to_datetime(data["dates"], format='%Y/%m/%d')
        data.set_index("dates", inplace = True)
    
        timeseries_IEOPW.loc[:, str(station)] = data.Qm3s

# Check the final time series:
timeseries_IEOPW.head() 

### 10. Iceland

In [ ]:
## Icelandic time series from OPW
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IS'
filenames = glob.glob(path + "/*.csv")

timeseries_IS = pd.DataFrame(index = pd.date_range('01-01-1950','12-31-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # First we read our time-series:
    data = pd.read_csv(filename, encoding='latin-1', engine='python', delimiter = ";")

    # Convert our column of dates to datetime format:
    data["date"] = data["DD"].astype(str) + "-" + data["MM"].astype(str)  + "-" + data["YYYY"].astype(str) 
    data["date"] = pd.to_datetime(data["date"], format='%d-%m-%Y')

    # Here we can set the index as the dates column:
    data.set_index("date", inplace = True)

    # First we can retrieve the station name:
    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 1)[1]
    # Remove ".csv" from the string
    namestation = namestation.replace(".csv", "")
    
    timeseries_IS.loc[:, int(namestation)] = data.qobs

# Check the final time series:
timeseries_IS.head() 

### 11. Italy

- Emilia-Romagna

In [ ]:
## Italian time series from Emilia-Romagna
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/EMI'
filenames = glob.glob(path + "/*.csv")

timeseries_ITEM = pd.DataFrame(index = pd.date_range('01-01-1950','12-31-2022', freq='D'))
network_ITEM = pd.DataFrame(np.empty((len(filenames),10)))

# Initialize counter
i = 0

# Loop through each filename using tqdm to track progress
for filename in tqdm.tqdm(filenames):
    
    # Read network file with specific encoding, skiprows, delimiter, and footer
    network_filename = pd.read_csv(filename, encoding='latin-1', skiprows=4, delimiter=",",  
                                   skipfooter=6, names=["Date_start", "dates", "Q_m3s"])

    # Convert 'dates' column to datetime format
    network_filename["dates"] = pd.to_datetime(network_filename["dates"].astype(str).str[:10])
    
    # Set 'dates' column as index
    network_filename.set_index("dates", inplace=True)
    
    # Get the number of lines in the file
    num_lines = len(network_filename)
    
    # Read info file with specific encoding, skiprows, delimiter, and footer
    info_filename = pd.read_csv(filename, encoding='latin-1', skiprows=num_lines + 4, delimiter=",",  
                                skipfooter=2)
    
    # Extract station information
    station = info_filename.iloc[0, 0]
    
    # Assign station info to the corresponding row in network_ITEM DataFrame
    network_ITEM.iloc[i, :] = info_filename.iloc[0, :]
    
    # Assign discharge data to the corresponding column in timeseries_ITEM DataFrame
    timeseries_ITEM.loc[:, station] = network_filename.Q_m3s
    
    # Increment counter
    i += 1
    
# Set column names of network_ITEM DataFrame based on info_filename
network_ITEM.columns = info_filename.columns

# Check the final time series:
timeseries_ITEM.head() 

- Umbria

In [ ]:
## Italian time series from Umbria
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/UMB'
filename = glob.glob(path + "/*.xlsx")
len(filename)

### Retrieve first the stations list:
timeseries = pd.read_excel(filename[0], skiprows=0, 
                          usecols = ['ID_SENSORE_DETTAGLIO', 'NOME_STAZIONE', 'LATITUDINE', 'LONGITUDINE', 'ANNO', 'MESE',
                                     'GIORNO', 'AVGDAY'])

timeseries["dates"] = timeseries['ANNO'].astype(str) + "-" + timeseries["MESE"].astype(str) + "-" + timeseries["GIORNO"].astype(str)
timeseries["dates"] = pd.to_datetime(timeseries["dates"], format='%Y-%m-%d')
timeseries.drop(["ANNO", "MESE", "GIORNO"], axis = 1, inplace = True)

names = ['Code', 'Name', 'Lat', 'Lon', "Q_m3_s", "dates"]

timeseries.columns = names

stations_list_ITUM = timeseries["Code"].unique().tolist()

### Retrieve and organize the time series:
# Create a DataFrame to store the final time series data with dates as index
timeseries_ITUM = pd.DataFrame(index=pd.date_range('01-01-1925', '12-31-2022', freq='D'))

# Create an empty DataFrame to store network information for Umbria
network_ITUM = pd.DataFrame(index=range(12), columns=['Code', 'Name', 'Lat', 'Lon'])

# Initialize counter
i = 0

# Loop through each station in the list, tqdm is used to track progress
for station in tqdm.tqdm(stations_list_ITUM):
    
    # Filter time series data for the current station
    timeseries_station = timeseries[timeseries.Code == station]
    
    # Set 'dates' column as index
    timeseries_station.set_index('dates', inplace=True)

    # Remove duplicate dates if any
    timeseries_station = timeseries_station.loc[timeseries_station.index.drop_duplicates(keep=False), :]
    
    # Assign discharge data to the corresponding column in timeseriesfinal DataFrame
    timeseries_ITUM.loc[:, station] = timeseries_station.Q_m3_s
    
    # Assign station information to the corresponding row in network_umbria DataFrame
    network_ITUM.iloc[i, :] = timeseries_station.iloc[0, [0, 1, 2, 3]]
    
    # Increment counter
    i += 1

# Check the final time series:
timeseries_ITUM.head() 

- Vale do Aosta

In [ ]:
## Italian time series from Vale D'aosta
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/VAL'
filenames = glob.glob(path + "/*.csv")

# Create a DataFrame to store the final time series data with dates as index
timeseries_ITVA = pd.DataFrame(index=pd.date_range('01-01-1978', '12-31-2022', freq='D'))

# Initialize counter
i = 0

# Loop through each filename in the list of filenames, tqdm is used to track progress
for filename in tqdm.tqdm(filenames):
    
    # Read time series data from the current file
    timeseries_station = pd.read_csv(filename, encoding='latin-1', skiprows=7, delimiter=";", header=None, 
                                      names=["dates", "Q_m3_s"], decimal=',')
    
    # Convert 'dates' column to datetime format
    timeseries_station["dates"] = pd.to_datetime(timeseries_station["dates"], format='%Y-%m-%d')
    
    # Set 'dates' column as index
    timeseries_station.set_index("dates", inplace=True)
    
    # Remove duplicate dates if any
    timeseries_station = timeseries_station.loc[timeseries_station.index.drop_duplicates(keep=False), :]
    
    # Read station information from the current file
    station = pd.read_csv(filename, encoding='latin-1', skiprows=2, delimiter=";", header=None, nrows=1)
    station = station.iloc[:, 0].str.replace('Stazione: ',' ')
    
    # Assign discharge data to the corresponding column in timeseriesfinal DataFrame
    timeseries_ITVA.loc[:, station[0]] = timeseries_station.Q_m3_s
    
    # Increment counter
    i += 1

# Check the final time series:
timeseries_ITVA.head()     

- Piemonte

In [ ]:
## Italian time series from Piemonte
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/PIE'
filenames = glob.glob(path + "/*.csv")

# Initialize a DataFrame to store the final time series data with dates as index:
timeseries_ITPI = pd.DataFrame(index=pd.date_range('01-01-1990', '12-31-2022', freq='D'))

# Initialize a counter variable:
i = 0

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # Extract the station name from the filename:
    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 1)[0]
    
    # Read the CSV file containing the time series data for the station:
    timeseries_station = pd.read_csv(filename, delimiter=";", encoding='latin-1', usecols=[0, 1], decimal=',')
    timeseries_station.columns = ["dates", "Q_m3_s"]
    
    # Convert the 'dates' column to datetime format and set it as index:
    timeseries_station["dates"] = pd.to_datetime(timeseries_station["dates"], format='%d/%m/%Y')
    timeseries_station.set_index("dates", inplace=True)
    
    # Store the time series data for the station in the final DataFrame:
    timeseries_ITPI.loc[:, namestation] = timeseries_station.Q_m3_s
    
    # Increment the counter:
    i = i + 1

# Check the final time series:
timeseries_ITPI.head()   


- Trento

In [ ]:
## Italian time series from Trento
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/TRE'
filenames = glob.glob(path + "/*.csv")

# Initialize a DataFrame to store the final time series data with dates as index:
timeseries_ITTR = pd.DataFrame(index=pd.date_range('10-01-1978', '12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in filenames:
    # Read the CSV file containing the time series data:
    timeseries_station = pd.read_csv(filename, delimiter=",", encoding='latin-1', skiprows=6, decimal='.')
    timeseries_station.columns = ["dates", "Q_m3_s"]
    
    # Convert the 'dates' column to datetime format and set it as index:
    timeseries_station["dates"] = pd.to_datetime(timeseries_station["dates"], format='%Y-%m-%d')
    timeseries_station.set_index("dates", inplace=True)

    # Resample the data to fill any missing dates and compute daily means:
    timeseries_station = timeseries_station.resample('D').mean()

    # Extract the station name from the file and store it:
    namestation = pd.read_csv(filename, delimiter=",", encoding='latin-1', skiprows=3, nrows=1)
    namestation = namestation.iloc[0, 1]
    
    # Store the time series data for the station in the final DataFrame:
    timeseries_ITTR.loc[:, namestation] = timeseries_station.Q_m3_s
    
# Check the final time series:
timeseries_ITTR.head() 

- Toscana

In [ ]:
## Italian time series from Toscana
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/TOS'
filenames = glob.glob(path + "/*.csv")

# Initialize a DataFrame to store the final time series data with dates as index:
timeseries_ITTO = pd.DataFrame(index=pd.date_range('01-01-1920', '09-30-2023', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # Read the CSV file containing the time series data:
    timeseries_station = pd.read_csv(filename, encoding='latin-1', skiprows=19, delimiter=";", 
                                     names=["dates", "Q_m3_s", "Val"], decimal=',')

    # Convert the 'dates' column to datetime format and set it as index:
    timeseries_station["dates"] = pd.to_datetime(timeseries_station["dates"], format='%d/%m/%Y')
    timeseries_station.set_index("dates", inplace=True)
    
    # Extract the station name from the file:
    station = pd.read_csv(filename, encoding='latin-1', skiprows=0, delimiter=";", nrows=1)
    station = station.iloc[:, 1]
    
    # Resample the data to fill any missing dates and compute daily means:
    timeseries_station = timeseries_station.resample('D').mean()
    
    # Store the time series data for the station in the final DataFrame:
    timeseries_ITTO.loc[:, station[0]] = timeseries_station.Q_m3_s
    
# Check the final time series:
timeseries_ITTO.head() 

- ISPRA

In [ ]:
## Italian time series from ISPRA
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/IT/ISPRA'
filenames = glob.glob(path + "/*.csv")

# Initialize a DataFrame to store the final time series data with dates as index:
timeseries_ITIS = pd.DataFrame(index=pd.date_range('1900-01-01', '2022-12-31', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # Read the CSV file containing the time series data:
    timeseries_filename = pd.read_csv(filename, encoding='latin-1', header=0, delimiter=",", 
                                      decimal=".", names=["dates", "Q_m3s", "Quality"])
    
    # Replace negative values in 'Q_m3s' column with NaN:
    timeseries_filename.Q_m3s[timeseries_filename.Q_m3s < 0] = np.nan
    
    # Convert the 'dates' column to datetime format:
    timeseries_filename["dates"] = pd.to_datetime(timeseries_filename["dates"], format='%Y-%m-%dT%H:%M:%S')
    
    # Set 'dates' column as index:
    timeseries_filename.set_index("dates", inplace=True)
    
    # Resample the data to daily intervals, taking the mean of each day:
    timeseries_filename = timeseries_filename.resample('D').mean()
    
    # Extract the station name from the filename:
    namestation = os.path.basename(filename)
    namestation = namestation.replace(".csv", "")
    namestation = namestation.replace("_", ":")
    
    # Store the time series data for the station in the final DataFrame:
    timeseries_ITIS.loc[:, namestation] = timeseries_filename.Q_m3s

# Check the final time series:
timeseries_ITIS.head() 

### 12. Poland

In [ ]:
# Polish daily streamflow files

# First you should upload the network information file
network_PL = pd.read_csv(r'data/streamflow/raw_data/PL/network/lista_stacji_hydro.csv', encoding='latin-1', header = None,
                        names = ["code", "name", "river", "col4"])

# Now we upload the streamflow data
path =r'data/streamflow/raw_data/PL'
filenames = glob.glob(path + "/*.csv")

# Here we create an empty dataframe to be populated with the streamflow data
timeseries_PL = pd.DataFrame(index = pd.date_range('01-01-1950','12-31-2022', freq='D'), columns = network_PL.code)

# We make one loop to cover each month avaialble in the dataset
for filename in tqdm.tqdm(filenames):
    
    # Now we read the data for the specific month
    timeseries_month = pd.read_csv(filename, encoding='latin-1', delimiter = ",", header = None, names = ["code", "name", "river", 
                                                                                                          "year_hydro", "month_hydro", 
                                                                                                          "day", "level", "flow", 
                                                                                                          "temperature", "month"])

    # Pay attention that we read the hydrological year (not the calendar), so we should convert it before
    # Apply the condition and update the 'year' column according to the calendar month (hydrological year starts in November)
    timeseries_month['year'] = timeseries_month['year_hydro']
    timeseries_month.loc[timeseries_month['month'] >= 11, 'year'] = timeseries_month['year_hydro'] - 1
    
    # Create a new column with the dates (converted to datetime)
    timeseries_month["date"] = pd.to_datetime(timeseries_month[['year', 'month', 'day']])

    # Here we retrieve the stations in this month (unique ones)
    stations_this_month = timeseries_month.code.unique().tolist()

    # Here we make a loop to retrieve the data for the specific month for each station
    for station in stations_this_month:
        timeseries_station = timeseries_month[timeseries_month.code == station]
        timeseries_station.set_index("date", inplace=True)
        timeseries_PL.loc[timeseries_station.index, station] = timeseries_station.flow

### 13. Portugal

In [ ]:
# Portuguese daily streamflow files
# First we can check our files within the folder. For Portugal, we can download the 
# data in groups of maximum (10 gauges) according to the record numbers, so the code takes
# care of the proper aggregation regardless of how the csvs are organized:
path =r'data/streamflow/raw_data/PT'
filenames = glob.glob(path + "/*.csv")

# Iterate through each file and process its contents:
i = 0
for filename in tqdm.tqdm(filenames):
    
    # There are some useless values at the end after a blank line. As it varies from file to file, we apply the following 
    # methodology to get rid of them:
    
    tsdf = pd.read_csv(filename, encoding='latin-1', skiprows=2, engine='python', skip_blank_lines=False)
    blank_df = tsdf.loc[tsdf.isnull().all(1)]
    if len(blank_df) > 0:
        first_blank_index = blank_df.index[0]
        tsdf = tsdf[:first_blank_index]
    
    # We also need to delete the first row, and all the columns that contain 'Unnamed':
    tsdf = tsdf.drop([0])
    tsdf = tsdf.drop(tsdf.columns[tsdf.columns.str.contains(pat = 'Unnamed')].tolist(), axis = 1)

    # Now we convert our dataset column that will be used as index to datetime:
    tsdf["dates"] = pd.to_datetime(tsdf['DATA'], format='%d/%m/%Y %H:%M')
    tsdf = tsdf.drop("DATA", axis = 1)
    tsdf.set_index('dates', inplace = True)

    # There are rows missing when there are gaps, therefore this is a nice way of overcame this and have our time-series 
    # range completed:
    timeseries = pd.DataFrame(index = pd.date_range('01-01-1900','12-31-2023', freq='D'), data = tsdf.astype('float'))
    
    # Now the concatatanion is made, and at the end we might have the total of streamflow gauges as columns:
    if i == 0:
        timeseries_PT = timeseries
    else:
        timeseries_PT = pd.concat([timeseries_PT, timeseries], axis=1)
    i = i + 1

# Check the final time series:
timeseries_PT.head()       

### 14. Slovenia

In [ ]:
## Slovenian time series from OPW
# First we can check our files within the folder:
path =r'data/streamflow/raw_data/SI'
filenames = glob.glob(path + "/*.xls")

timeseries_SI = pd.DataFrame(index = pd.date_range('01-01-1950','12-31-2022', freq='D'))

# Iterate through each file and process its contents:
for filename in tqdm.tqdm(filenames):
    # First we read our time-series:
    data = pd.read_excel(filename, sheet_name = 0, usecols = ["Datum", "pretok (m3/s)"])

    # We define new column names:
    names = ['dates', 'Qm3s']
    data.columns = names

    # Convert our column of dates to datetime format:
    data["dates"] = pd.to_datetime(data["dates"], format='%d.%m.%Y')

    # Here we can set the index as the dates column:
    data.set_index("dates", inplace = True)

    # First we can retrieve the station name:
    namestation = os.path.basename(filename)
    namestation = namestation.split("_", 4)[1]
    timeseries_SI.loc[:, int(namestation)] = data.Qm3s

# Check the final time series:
timeseries_SI.head()       

## Exporting the concatenated data

In [ ]:
# After the concatenation of the desired datasets, users can simply export the file as:
PATH_OUTPUT = "PATH_TO_WHERE_YOU_WANT_TO_EXPORT"
timeseries_AT.to_csv(PATH_OUTPUT+"timeseries_AT.csv")

# End